# 🛡️ Lección 8: Ética, Robots.txt y Mejores Prácticas

## 🎯 Objetivos Finales

- Entender los aspectos legales y éticos del web scraping
- Implementar respeto por robots.txt y políticas
- Dominar rate limiting y cortesía web
- Crear sistemas de scraping responsables
- Aplicar mejores prácticas industriales
- Desarrollar scrapers sostenibles y éticos

In [ ]:
import requests
import time
import urllib.robotparser
from urllib.parse import urljoin, urlparse
import logging
from datetime import datetime, timedelta
from collections import defaultdict
import re
import json
from typing import Dict, List, Optional, Tuple

print("🛡️ ÉTICA Y MEJORES PRÁCTICAS EN WEB SCRAPING")
print("=" * 55)
print("✅ Módulos importados correctamente")
print("📖 Preparados para aprender scraping responsable")

## ⚖️ Aspectos Legales y Éticos

### 📜 Marco Legal del Web Scraping

El web scraping existe en una zona gris legal que depende de varios factores:

#### ✅ Generalmente Legal:
- Datos públicos sin autenticación
- Información factual (precios, horarios)
- Datos para uso personal/educativo
- Respeto a robots.txt y términos de servicio
- Rate limiting apropiado

#### ❌ Potencialmente Problemático:
- Datos detrás de autenticación
- Contenido con copyright
- Información personal/privada
- Violación de términos de servicio
- Sobrecarga de servidores
- Uso comercial no autorizado

In [ ]:
class EthicalScrapingChecker:
    """Verificador de prácticas éticas de scraping"""
    
    def __init__(self):
        self.ethical_guidelines = {
            'respect_robots_txt': True,
            'rate_limit_enabled': True,
            'user_agent_identified': True,
            'public_data_only': True,
            'terms_of_service_reviewed': False,  # Usuario debe confirmar
            'commercial_use_authorized': False,  # Usuario debe confirmar
            'data_minimization': True,  # Solo extraer lo necesario
            'privacy_respected': True   # No datos personales
        }
        
        self.risk_factors = {
            'authentication_required': 'high',
            'personal_data_present': 'high',
            'copyright_content': 'medium',
            'high_frequency_requests': 'medium',
            'commercial_competitor': 'medium',
            'no_robots_txt': 'low'
        }
    
    def check_url_ethics(self, url: str, purpose: str = 'research') -> Dict[str, any]:
        """Evaluar aspectos éticos de scrapear una URL"""
        domain = urlparse(url).netloc
        
        ethics_report = {
            'url': url,
            'domain': domain,
            'purpose': purpose,
            'timestamp': datetime.now().isoformat(),
            'ethical_score': 0,
            'recommendations': [],
            'warnings': [],
            'legal_considerations': []
        }
        
        # Verificar robots.txt
        robots_check = self.check_robots_txt(url)
        if robots_check['allowed']:
            ethics_report['ethical_score'] += 20
        else:
            ethics_report['warnings'].append(f"🚫 Robots.txt prohíbe el acceso: {robots_check['rule']}")
            ethics_report['ethical_score'] -= 30
        
        # Verificar si es un sitio público
        if 'login' in url.lower() or 'auth' in url.lower():
            ethics_report['warnings'].append("🔐 URL parece requerir autenticación")
            ethics_report['legal_considerations'].append("Datos detrás de autenticación pueden requerir autorización")
            ethics_report['ethical_score'] -= 25
        else:
            ethics_report['ethical_score'] += 15
        
        # Evaluar propósito
        purpose_scores = {
            'research': 20,
            'education': 20,
            'personal': 15,
            'commercial': -10,
            'competitive_analysis': -5
        }
        
        ethics_report['ethical_score'] += purpose_scores.get(purpose, 0)
        
        if purpose == 'commercial':
            ethics_report['legal_considerations'].append(
                "Uso comercial requiere revisar términos de servicio y posibles licencias"
            )
        
        # Verificar dominio conocido
        sensitive_domains = ['facebook.com', 'linkedin.com', 'twitter.com', 'instagram.com']
        if any(sensitive in domain for sensitive in sensitive_domains):
            ethics_report['warnings'].append("⚠️ Dominio con políticas estrictas sobre scraping")
            ethics_report['legal_considerations'].append(
                "Redes sociales tienen términos de servicio muy restrictivos"
            )
            ethics_report['ethical_score'] -= 15
        
        # Generar recomendaciones
        if ethics_report['ethical_score'] >= 50:
            ethics_report['recommendations'].append("✅ Práctica generalmente ética")
        elif ethics_report['ethical_score'] >= 20:
            ethics_report['recommendations'].append("⚠️ Proceder con precaución")
        else:
            ethics_report['recommendations'].append("❌ Alto riesgo ético/legal")
        
        # Recomendaciones generales
        ethics_report['recommendations'].extend([
            "📖 Revisar términos de servicio del sitio",
            "🤖 Respetar robots.txt",
            "⏱️ Implementar rate limiting",
            "🆔 Usar User-Agent identificable",
            "📊 Extraer solo datos necesarios",
            "🔒 Evitar datos personales/privados"
        ])
        
        return ethics_report
    
    def check_robots_txt(self, url: str) -> Dict[str, any]:
        """Verificar robots.txt"""
        try:
            parsed_url = urlparse(url)
            robots_url = f"{parsed_url.scheme}://{parsed_url.netloc}/robots.txt"
            
            rp = urllib.robotparser.RobotFileParser()
            rp.set_url(robots_url)
            rp.read()
            
            allowed = rp.can_fetch('*', url)
            
            return {
                'robots_url': robots_url,
                'allowed': allowed,
                'rule': f"{'Permitido' if allowed else 'No permitido'} para user-agent '*'"
            }
            
        except Exception as e:
            return {
                'robots_url': 'N/A',
                'allowed': True,  # Si no hay robots.txt, asumir permitido
                'rule': f'No se pudo leer robots.txt: {e}'
            }

# Ejemplo de evaluación ética
print("⚖️ EVALUACIÓN ÉTICA DE URLs")
print("=" * 35)

ethics_checker = EthicalScrapingChecker()

# URLs de ejemplo para evaluar
test_urls = [
    ('http://quotes.toscrape.com', 'education'),
    ('https://example-store.com/products', 'research'),
    ('https://facebook.com/profile/123', 'commercial'),
    ('https://api.private-site.com/data', 'competitive_analysis')
]

for url, purpose in test_urls:
    print(f"\n🔍 Evaluando: {url} (Propósito: {purpose})")
    
    report = ethics_checker.check_url_ethics(url, purpose)
    
    print(f"📊 Puntuación ética: {report['ethical_score']}/100")
    
    if report['warnings']:
        print("⚠️ Advertencias:")
        for warning in report['warnings']:
            print(f"   {warning}")
    
    print(f"💡 Recomendación principal: {report['recommendations'][0]}")
    
    if report['legal_considerations']:
        print(f"⚖️ Consideración legal: {report['legal_considerations'][0]}")

## 🤖 Robots.txt: Respeto y Cumplimiento

In [ ]:
class RobotsTxtManager:
    """Gestor completo de robots.txt"""
    
    def __init__(self, user_agent: str = '*'):
        self.user_agent = user_agent
        self.robots_cache = {}  # Cache de robots.txt
        self.cache_expiry = 3600  # 1 hora
    
    def get_robots_txt(self, domain: str) -> Optional[urllib.robotparser.RobotFileParser]:
        """Obtener y cachear robots.txt"""
        current_time = time.time()
        
        # Verificar cache
        if domain in self.robots_cache:
            cached_data = self.robots_cache[domain]
            if current_time - cached_data['timestamp'] < self.cache_expiry:
                return cached_data['parser']
        
        try:
            robots_url = f"https://{domain}/robots.txt"
            
            rp = urllib.robotparser.RobotFileParser()
            rp.set_url(robots_url)
            rp.read()
            
            # Cachear resultado
            self.robots_cache[domain] = {
                'parser': rp,
                'timestamp': current_time
            }
            
            return rp
            
        except Exception as e:
            print(f"Error leyendo robots.txt de {domain}: {e}")
            return None
    
    def can_fetch(self, url: str) -> Tuple[bool, str]:
        """Verificar si se puede hacer scraping de una URL"""
        parsed_url = urlparse(url)
        domain = parsed_url.netloc
        
        rp = self.get_robots_txt(domain)
        
        if rp is None:
            return True, "No se pudo obtener robots.txt - asumiendo permitido"
        
        allowed = rp.can_fetch(self.user_agent, url)
        
        if allowed:
            return True, "Permitido por robots.txt"
        else:
            return False, "Prohibido por robots.txt"
    
    def get_crawl_delay(self, domain: str) -> Optional[float]:
        """Obtener delay recomendado de robots.txt"""
        rp = self.get_robots_txt(domain)
        
        if rp is None:
            return None
        
        delay = rp.crawl_delay(self.user_agent)
        return delay
    
    def get_sitemaps(self, domain: str) -> List[str]:
        """Obtener sitemaps listados en robots.txt"""
        rp = self.get_robots_txt(domain)
        
        if rp is None:
            return []
        
        try:
            return rp.site_maps() or []
        except AttributeError:
            return []
    
    def analyze_robots_txt(self, domain: str) -> Dict[str, any]:
        """Análisis completo de robots.txt"""
        analysis = {
            'domain': domain,
            'robots_url': f"https://{domain}/robots.txt",
            'accessible': False,
            'user_agents': [],
            'disallowed_paths': [],
            'allowed_paths': [],
            'crawl_delay': None,
            'sitemaps': [],
            'raw_content': ''
        }
        
        try:
            # Obtener contenido raw
            response = requests.get(analysis['robots_url'], timeout=10)
            response.raise_for_status()
            
            analysis['accessible'] = True
            analysis['raw_content'] = response.text
            
            # Parsear contenido
            lines = response.text.split('\n')
            current_user_agent = None
            
            for line in lines:
                line = line.strip()
                if not line or line.startswith('#'):
                    continue
                
                if line.lower().startswith('user-agent:'):
                    current_user_agent = line.split(':', 1)[1].strip()
                    if current_user_agent not in analysis['user_agents']:
                        analysis['user_agents'].append(current_user_agent)
                
                elif line.lower().startswith('disallow:'):
                    path = line.split(':', 1)[1].strip()
                    if path and path not in analysis['disallowed_paths']:
                        analysis['disallowed_paths'].append(path)
                
                elif line.lower().startswith('allow:'):
                    path = line.split(':', 1)[1].strip()
                    if path and path not in analysis['allowed_paths']:
                        analysis['allowed_paths'].append(path)
                
                elif line.lower().startswith('crawl-delay:'):
                    try:
                        delay = float(line.split(':', 1)[1].strip())
                        analysis['crawl_delay'] = delay
                    except ValueError:
                        pass
                
                elif line.lower().startswith('sitemap:'):
                    sitemap = line.split(':', 1)[1].strip()
                    if sitemap not in analysis['sitemaps']:
                        analysis['sitemaps'].append(sitemap)
        
        except Exception as e:
            analysis['error'] = str(e)
        
        return analysis

# Ejemplo de análisis de robots.txt
print("🤖 ANÁLISIS DE ROBOTS.TXT")
print("=" * 30)

robots_manager = RobotsTxtManager(user_agent='EthicalScraper/1.0')

# Analizar varios dominios
test_domains = ['quotes.toscrape.com', 'httpbin.org', 'example.com']

for domain in test_domains:
    print(f"\n🌐 Analizando robots.txt de {domain}:")
    
    analysis = robots_manager.analyze_robots_txt(domain)
    
    if analysis['accessible']:
        print(f"✅ Robots.txt accesible")
        print(f"👥 User agents: {len(analysis['user_agents'])}")
        print(f"🚫 Rutas prohibidas: {len(analysis['disallowed_paths'])}")
        print(f"✅ Rutas permitidas: {len(analysis['allowed_paths'])}")
        
        if analysis['crawl_delay']:
            print(f"⏱️ Delay recomendado: {analysis['crawl_delay']}s")
        
        if analysis['sitemaps']:
            print(f"🗺️ Sitemaps: {len(analysis['sitemaps'])}")
        
        # Verificar URLs específicas
        test_url = f"https://{domain}/"
        can_fetch, reason = robots_manager.can_fetch(test_url)
        print(f"🔍 Scraping de raíz: {'✅ Permitido' if can_fetch else '❌ Prohibido'} ({reason})")
    
    else:
        print(f"❌ No se pudo acceder a robots.txt")
        if 'error' in analysis:
            print(f"Error: {analysis['error']}")

## ⏱️ Rate Limiting y Cortesía Web

In [ ]:
class PoliteWebScraper:
    """Scraper que implementa todas las mejores prácticas de cortesía"""
    
    def __init__(self, user_agent: str, base_delay: float = 1.0):
        self.user_agent = user_agent
        self.base_delay = base_delay
        
        # Gestión de delays por dominio
        self.domain_delays = defaultdict(lambda: base_delay)
        self.last_requests = defaultdict(float)
        self.request_counts = defaultdict(int)
        
        # Configuración de cortesía
        self.politeness_config = {
            'max_requests_per_minute': 30,
            'max_requests_per_hour': 1000,
            'adaptive_delay': True,
            'respect_server_load': True,
            'exponential_backoff': True
        }
        
        # Estadísticas
        self.stats = {
            'requests_made': 0,
            'total_delay_time': 0,
            'errors_encountered': 0,
            'domains_accessed': set(),
            'start_time': time.time()
        }
        
        # Robots.txt manager
        self.robots_manager = RobotsTxtManager(user_agent=user_agent)
        
        # Setup logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
    
    def calculate_delay(self, domain: str, response_time: float = None, status_code: int = 200) -> float:
        """Calcular delay apropiado basado en múltiples factores"""
        base_delay = self.domain_delays[domain]
        
        # Factor 1: Robots.txt crawl-delay
        robots_delay = self.robots_manager.get_crawl_delay(domain)
        if robots_delay:
            base_delay = max(base_delay, robots_delay)
        
        # Factor 2: Carga del servidor (basado en tiempo de respuesta)
        if response_time and self.politeness_config['respect_server_load']:
            if response_time > 5.0:  # Servidor lento
                base_delay *= 2.0
            elif response_time > 2.0:  # Servidor moderadamente lento
                base_delay *= 1.5
        
        # Factor 3: Códigos de estado (backoff exponencial)
        if self.politeness_config['exponential_backoff']:
            if status_code == 429:  # Too Many Requests
                base_delay *= 4.0
            elif status_code >= 500:  # Server errors
                base_delay *= 2.0
        
        # Factor 4: Frecuencia de requests
        current_time = time.time()
        requests_this_minute = self.request_counts[f"{domain}_minute"]
        
        if requests_this_minute > self.politeness_config['max_requests_per_minute']:
            base_delay *= 3.0  # Slow down significantly
        
        # Factor 5: Delay adaptativo
        if self.politeness_config['adaptive_delay']:
            # Aumentar delay gradualmente durante la sesión
            session_duration = current_time - self.stats['start_time']
            if session_duration > 3600:  # Después de 1 hora
                base_delay *= 1.2
        
        return min(base_delay, 30.0)  # Máximo 30 segundos
    
    def wait_for_politeness(self, domain: str, response_time: float = None, status_code: int = 200):
        """Esperar tiempo apropiado entre requests"""
        current_time = time.time()
        last_request_time = self.last_requests.get(domain, 0)
        
        # Calcular delay necesario
        required_delay = self.calculate_delay(domain, response_time, status_code)
        
        # Tiempo transcurrido desde último request
        time_since_last = current_time - last_request_time
        
        # Esperar si es necesario
        if time_since_last < required_delay:
            wait_time = required_delay - time_since_last
            
            self.logger.info(f"⏱️ Esperando {wait_time:.2f}s para {domain} (cortesía web)")
            
            time.sleep(wait_time)
            self.stats['total_delay_time'] += wait_time
        
        # Actualizar registros
        self.last_requests[domain] = time.time()
        self.update_request_counts(domain)
    
    def update_request_counts(self, domain: str):
        """Actualizar contadores de requests"""
        current_time = time.time()
        
        # Limpiar contadores antiguos
        minute_key = f"{domain}_minute"
        hour_key = f"{domain}_hour"
        
        # Resetear contadores si ha pasado el tiempo
        if current_time - self.last_requests.get(f"{minute_key}_reset", 0) > 60:
            self.request_counts[minute_key] = 0
            self.last_requests[f"{minute_key}_reset"] = current_time
        
        if current_time - self.last_requests.get(f"{hour_key}_reset", 0) > 3600:
            self.request_counts[hour_key] = 0
            self.last_requests[f"{hour_key}_reset"] = current_time
        
        # Incrementar contadores
        self.request_counts[minute_key] += 1
        self.request_counts[hour_key] += 1
    
    def polite_request(self, url: str, **kwargs) -> Optional[requests.Response]:
        """Realizar request con todas las consideraciones de cortesía"""
        domain = urlparse(url).netloc
        
        # Verificar robots.txt
        can_fetch, reason = self.robots_manager.can_fetch(url)
        if not can_fetch:
            self.logger.warning(f"🚫 Robots.txt prohíbe el acceso a {url}: {reason}")
            return None
        
        # Verificar límites de rate
        if not self.check_rate_limits(domain):
            self.logger.warning(f"⏱️ Límites de rate excedidos para {domain}")
            return None
        
        try:
            # Esperar tiempo de cortesía
            self.wait_for_politeness(domain)
            
            # Configurar headers apropiados
            headers = kwargs.get('headers', {})
            headers.update({
                'User-Agent': self.user_agent,
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.5',
                'Accept-Encoding': 'gzip, deflate',
                'Connection': 'keep-alive',
                'Upgrade-Insecure-Requests': '1'
            })
            kwargs['headers'] = headers
            
            # Realizar request
            start_time = time.time()
            response = requests.get(url, timeout=30, **kwargs)
            response_time = time.time() - start_time
            
            # Actualizar estadísticas
            self.stats['requests_made'] += 1
            self.stats['domains_accessed'].add(domain)
            
            # Log del request
            self.logger.info(
                f"📡 {response.status_code} {url} ({response_time:.2f}s, {len(response.content)} bytes)"
            )
            
            # Actualizar delay para próximo request
            self.wait_for_politeness(domain, response_time, response.status_code)
            
            return response
            
        except requests.RequestException as e:
            self.stats['errors_encountered'] += 1
            self.logger.error(f"❌ Error en request a {url}: {e}")
            
            # Aumentar delay después de error
            self.domain_delays[domain] *= 1.5
            
            return None
    
    def check_rate_limits(self, domain: str) -> bool:
        """Verificar si se pueden hacer más requests"""
        minute_count = self.request_counts.get(f"{domain}_minute", 0)
        hour_count = self.request_counts.get(f"{domain}_hour", 0)
        
        if minute_count >= self.politeness_config['max_requests_per_minute']:
            return False
        
        if hour_count >= self.politeness_config['max_requests_per_hour']:
            return False
        
        return True
    
    def get_politeness_report(self) -> Dict[str, any]:
        """Obtener reporte de cortesía"""
        current_time = time.time()
        session_duration = current_time - self.stats['start_time']
        
        return {
            'session_duration': session_duration,
            'requests_made': self.stats['requests_made'],
            'domains_accessed': len(self.stats['domains_accessed']),
            'total_delay_time': self.stats['total_delay_time'],
            'average_delay': self.stats['total_delay_time'] / max(self.stats['requests_made'], 1),
            'requests_per_minute': self.stats['requests_made'] / (session_duration / 60),
            'errors_encountered': self.stats['errors_encountered'],
            'politeness_ratio': self.stats['total_delay_time'] / session_duration * 100,
            'domains_list': list(self.stats['domains_accessed'])
        }

# Ejemplo de scraping cortés
print("⏱️ SCRAPING CORTÉS Y RESPONSABLE")
print("=" * 40)

# Crear scraper cortés
polite_scraper = PoliteWebScraper(
    user_agent='EthicalScraper/1.0 (+http://example.com/bot-info)',
    base_delay=2.0
)

# URLs de ejemplo
test_urls = [
    'http://httpbin.org/delay/1',
    'http://httpbin.org/status/200',
    'http://httpbin.org/json'
]

print("🌐 Realizando requests corteses...")
responses = []

for url in test_urls:
    print(f"\n📡 Requesting: {url}")
    response = polite_scraper.polite_request(url)
    
    if response:
        responses.append({
            'url': url,
            'status': response.status_code,
            'size': len(response.content)
        })
        print(f"✅ Success: {response.status_code} ({len(response.content)} bytes)")
    else:
        print(f"❌ Failed to fetch {url}")

# Reporte de cortesía
report = polite_scraper.get_politeness_report()

print(f"\n📊 REPORTE DE CORTESÍA:")
print(f"  ⏱️ Duración sesión: {report['session_duration']:.2f}s")
print(f"  📡 Requests realizados: {report['requests_made']}")
print(f"  🌐 Dominios accedidos: {report['domains_accessed']}")
print(f"  ⏳ Tiempo total de espera: {report['total_delay_time']:.2f}s")
print(f"  📊 Delay promedio: {report['average_delay']:.2f}s")
print(f"  🎯 Requests por minuto: {report['requests_per_minute']:.2f}")
print(f"  🛡️ Ratio de cortesía: {report['politeness_ratio']:.1f}%")
print(f"  ❌ Errores: {report['errors_encountered']}")

## 🎯 Framework Completo de Scraping Ético

In [ ]:
class EthicalScrapingFramework:
    """Framework completo para web scraping ético y responsable"""
    
    def __init__(self, project_name: str, contact_info: str):
        self.project_name = project_name
        self.contact_info = contact_info
        
        # Configurar User-Agent identificable
        self.user_agent = f"{project_name}/1.0 (+{contact_info})"
        
        # Componentes del framework
        self.ethics_checker = EthicalScrapingChecker()
        self.polite_scraper = PoliteWebScraper(self.user_agent)
        
        # Configuración ética
        self.ethical_config = {
            'max_pages_per_site': 1000,
            'max_concurrent_requests': 1,  # Conservador
            'minimum_delay': 3.0,  # 3 segundos mínimo
            'respect_robots_txt': True,
            'log_all_requests': True,
            'data_minimization': True,  # Solo extraer datos necesarios
            'auto_detect_limits': True,
            'stop_on_errors': True
        }
        
        # Log de auditoría
        self.audit_log = {
            'session_id': f"{project_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            'start_time': datetime.now().isoformat(),
            'requests_log': [],
            'ethics_checks': [],
            'errors_log': [],
            'data_extracted': 0
        }
        
        # Setup logging
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(f'{project_name}_ethical_scraping.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
        
        self.logger.info(f"🛡️ Iniciando framework de scraping ético para '{project_name}'")
    
    def pre_scraping_check(self, urls: List[str], purpose: str) -> Dict[str, any]:
        """Verificación ética previa al scraping"""
        self.logger.info(f"🔍 Realizando verificación ética previa para {len(urls)} URLs")
        
        check_results = {
            'total_urls': len(urls),
            'ethical_urls': 0,
            'problematic_urls': 0,
            'blocked_urls': [],
            'warnings': [],
            'approved_urls': [],
            'overall_ethics_score': 0
        }
        
        total_score = 0
        
        for url in urls:
            ethics_report = self.ethics_checker.check_url_ethics(url, purpose)
            
            # Log en auditoría
            self.audit_log['ethics_checks'].append({
                'url': url,
                'score': ethics_report['ethical_score'],
                'warnings': ethics_report['warnings'],
                'timestamp': datetime.now().isoformat()
            })
            
            total_score += ethics_report['ethical_score']
            
            if ethics_report['ethical_score'] >= 50:
                check_results['ethical_urls'] += 1
                check_results['approved_urls'].append(url)
            elif ethics_report['ethical_score'] >= 20:
                check_results['warnings'].append(f"⚠️ {url}: Proceder con precaución")
                check_results['approved_urls'].append(url)
            else:
                check_results['problematic_urls'] += 1
                check_results['blocked_urls'].append(url)
                self.logger.warning(f"🚫 URL bloqueada por razones éticas: {url}")
        
        check_results['overall_ethics_score'] = total_score / len(urls) if urls else 0
        
        # Decisión final
        if check_results['overall_ethics_score'] < 30:
            self.logger.error(f"❌ Proyecto rechazado: Score ético muy bajo ({check_results['overall_ethics_score']:.1f})")
            check_results['approved'] = False
        else:
            self.logger.info(f"✅ Proyecto aprobado con score ético de {check_results['overall_ethics_score']:.1f}")
            check_results['approved'] = True
        
        return check_results
    
    def ethical_scrape(self, url: str, extract_function=None) -> Optional[Dict[str, any]]:
        """Realizar scraping ético de una URL"""
        self.logger.info(f"🌐 Iniciando scraping ético de {url}")
        
        start_time = time.time()
        
        try:
            # Realizar request cortés
            response = self.polite_scraper.polite_request(url)
            
            if not response:
                self.logger.error(f"❌ No se pudo obtener respuesta de {url}")
                return None
            
            # Extraer datos si se proporciona función
            extracted_data = None
            if extract_function:
                try:
                    extracted_data = extract_function(response)
                    if extracted_data:
                        self.audit_log['data_extracted'] += 1
                except Exception as e:
                    self.logger.error(f"❌ Error extrayendo datos de {url}: {e}")
            
            # Log de auditoría del request
            request_log = {
                'url': url,
                'timestamp': datetime.now().isoformat(),
                'status_code': response.status_code,
                'response_size': len(response.content),
                'response_time': time.time() - start_time,
                'user_agent': self.user_agent,
                'data_extracted': extracted_data is not None
            }
            
            self.audit_log['requests_log'].append(request_log)
            
            return {
                'response': response,
                'data': extracted_data,
                'metadata': request_log
            }
            
        except Exception as e:
            self.logger.error(f"❌ Error en scraping ético de {url}: {e}")
            
            # Log de error
            self.audit_log['errors_log'].append({
                'url': url,
                'error': str(e),
                'timestamp': datetime.now().isoformat()
            })
            
            return None
    
    def batch_ethical_scrape(self, urls: List[str], extract_function=None, max_pages: int = None) -> List[Dict[str, any]]:
        """Scraping ético en lote"""
        if max_pages:
            urls = urls[:max_pages]
        
        self.logger.info(f"🚀 Iniciando scraping ético en lote de {len(urls)} URLs")
        
        results = []
        
        for i, url in enumerate(urls, 1):
            self.logger.info(f"📄 Procesando {i}/{len(urls)}: {url}")
            
            result = self.ethical_scrape(url, extract_function)
            
            if result:
                results.append(result)
            
            # Verificar si debemos parar por errores
            if self.ethical_config['stop_on_errors'] and len(self.audit_log['errors_log']) > 5:
                self.logger.warning("⏹️ Deteniendo scraping por exceso de errores")
                break
            
            # Progress update cada 10 URLs
            if i % 10 == 0:
                success_rate = len(results) / i * 100
                self.logger.info(f"📊 Progreso: {i}/{len(urls)} URLs ({success_rate:.1f}% éxito)")
        
        self.logger.info(f"✅ Scraping en lote completado: {len(results)} URLs exitosas")
        return results
    
    def generate_compliance_report(self) -> Dict[str, any]:
        """Generar reporte de cumplimiento ético"""
        current_time = datetime.now()
        
        # Estadísticas del scraper cortés
        politeness_stats = self.polite_scraper.get_politeness_report()
        
        compliance_report = {
            'project_info': {
                'name': self.project_name,
                'contact': self.contact_info,
                'user_agent': self.user_agent,
                'session_id': self.audit_log['session_id']
            },
            'session_summary': {
                'start_time': self.audit_log['start_time'],
                'end_time': current_time.isoformat(),
                'duration_hours': (current_time - datetime.fromisoformat(self.audit_log['start_time'])).total_seconds() / 3600,
                'total_requests': len(self.audit_log['requests_log']),
                'successful_requests': len([r for r in self.audit_log['requests_log'] if r['status_code'] == 200]),
                'failed_requests': len(self.audit_log['errors_log']),
                'data_points_extracted': self.audit_log['data_extracted']
            },
            'ethical_compliance': {
                'ethics_checks_performed': len(self.audit_log['ethics_checks']),
                'average_ethics_score': sum(c['score'] for c in self.audit_log['ethics_checks']) / len(self.audit_log['ethics_checks']) if self.audit_log['ethics_checks'] else 0,
                'robots_txt_respected': True,  # Nuestro framework siempre respeta robots.txt
                'rate_limiting_applied': True,
                'average_delay_seconds': politeness_stats.get('average_delay', 0),
                'politeness_ratio': politeness_stats.get('politeness_ratio', 0)
            },
            'performance_metrics': politeness_stats,
            'errors_encountered': self.audit_log['errors_log'],
            'recommendations': self.generate_recommendations()
        }
        
        return compliance_report
    
    def generate_recommendations(self) -> List[str]:
        """Generar recomendaciones basadas en la sesión"""
        recommendations = []
        
        politeness_stats = self.polite_scraper.get_politeness_report()
        
        # Recomendaciones basadas en métricas
        if politeness_stats.get('requests_per_minute', 0) > 10:
            recommendations.append("⚠️ Considerar reducir la velocidad de requests")
        
        if politeness_stats.get('errors_encountered', 0) > 5:
            recommendations.append("🔧 Revisar manejo de errores y reintentos")
        
        if len(self.audit_log['errors_log']) > len(self.audit_log['requests_log']) * 0.1:
            recommendations.append("📊 Alta tasa de errores - verificar targets")
        
        # Recomendaciones generales
        recommendations.extend([
            "✅ Continuar respetando robots.txt",
            "⏱️ Mantener delays apropiados",
            "📝 Documentar propósito de scraping",
            "🔄 Revisar periódicamente términos de servicio"
        ])
        
        return recommendations
    
    def save_compliance_report(self, filename: str = None) -> str:
        """Guardar reporte de cumplimiento"""
        if not filename:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            filename = f'{self.project_name}_compliance_report_{timestamp}.json'
        
        report = self.generate_compliance_report()
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(report, f, indent=2, default=str)
        
        self.logger.info(f"📄 Reporte de cumplimiento guardado: {filename}")
        return filename

# Función de extracción de ejemplo
def extract_quote_data(response):
    """Extraer datos de quotes.toscrape.com"""
    from bs4 import BeautifulSoup
    
    soup = BeautifulSoup(response.content, 'html.parser')
    quotes = []
    
    for quote_elem in soup.select('.quote'):
        quote_data = {
            'text': quote_elem.select_one('.text').text if quote_elem.select_one('.text') else '',
            'author': quote_elem.select_one('.author').text if quote_elem.select_one('.author') else '',
            'tags': [tag.text for tag in quote_elem.select('.tag')]
        }
        quotes.append(quote_data)
    
    return quotes

# Ejemplo de uso del framework completo
print("🎯 FRAMEWORK COMPLETO DE SCRAPING ÉTICO")
print("=" * 50)

# Crear framework
framework = EthicalScrapingFramework(
    project_name="EthicalQuoteScraper",
    contact_info="http://example.com/contact"
)

# URLs de ejemplo
test_urls = [
    'http://quotes.toscrape.com/',
    'http://quotes.toscrape.com/page/2/'
]

# Verificación ética previa
print("\n🔍 Realizando verificación ética previa...")
ethics_check = framework.pre_scraping_check(test_urls, 'education')

print(f"📊 Resultado de verificación ética:")
print(f"  ✅ URLs aprobadas: {ethics_check['ethical_urls']}/{ethics_check['total_urls']}")
print(f"  ⚠️ URLs problemáticas: {ethics_check['problematic_urls']}")
print(f"  🎯 Score ético general: {ethics_check['overall_ethics_score']:.1f}/100")
print(f"  📋 Proyecto aprobado: {'Sí' if ethics_check['approved'] else 'No'}")

if ethics_check['approved']:
    print("\n🚀 Iniciando scraping ético...")
    
    # Realizar scraping ético
    results = framework.batch_ethical_scrape(
        ethics_check['approved_urls'], 
        extract_function=extract_quote_data,
        max_pages=2
    )
    
    print(f"\n📊 Resultados del scraping:")
    print(f"  📄 Páginas procesadas: {len(results)}")
    
    total_quotes = sum(len(r['data']) for r in results if r['data'])
    print(f"  💬 Citas extraídas: {total_quotes}")
    
    # Generar reporte de cumplimiento
    print("\n📋 Generando reporte de cumplimiento...")
    compliance_report = framework.generate_compliance_report()
    
    print(f"\n🛡️ REPORTE DE CUMPLIMIENTO ÉTICO:")
    print(f"  📊 Requests totales: {compliance_report['session_summary']['total_requests']}")
    print(f"  ✅ Requests exitosos: {compliance_report['session_summary']['successful_requests']}")
    print(f"  ⏱️ Delay promedio: {compliance_report['ethical_compliance']['average_delay_seconds']:.2f}s")
    print(f"  🎯 Score ético promedio: {compliance_report['ethical_compliance']['average_ethics_score']:.1f}")
    print(f"  🛡️ Ratio de cortesía: {compliance_report['ethical_compliance']['politeness_ratio']:.1f}%")
    
    # Guardar reporte
    report_file = framework.save_compliance_report()
    print(f"\n💾 Reporte completo guardado en: {report_file}")
    
    print("\n💡 Recomendaciones principales:")
    for rec in compliance_report['recommendations'][:3]:
        print(f"  {rec}")

else:
    print("❌ Proyecto no aprobado por razones éticas")

print("\n✅ Framework de scraping ético completado")

## 📋 Checklist de Mejores Prácticas

In [ ]:
def generate_best_practices_checklist():
    """Generar checklist completo de mejores prácticas"""
    
    checklist = {
        "📖 Aspectos Legales y Éticos": [
            "✅ Revisar términos de servicio del sitio web",
            "✅ Verificar que los datos sean públicamente accesibles",
            "✅ Evitar scraping de datos personales o privados",
            "✅ Obtener autorización para uso comercial si es necesario",
            "✅ Respetar derechos de autor y propiedad intelectual",
            "✅ Documentar el propósito del scraping",
            "✅ Considerar alternativas como APIs oficiales"
        ],
        
        "🤖 Respeto a Robots.txt": [
            "✅ Verificar y respetar robots.txt antes de scrapear",
            "✅ Implementar parser de robots.txt en el código",
            "✅ Respetar directivas de Crawl-delay",
            "✅ Revisar sitemaps indicados en robots.txt",
            "✅ Actualizar cache de robots.txt periódicamente"
        ],
        
        "⏱️ Rate Limiting y Cortesía": [
            "✅ Implementar delays entre requests (mínimo 1-3 segundos)",
            "✅ Usar delays variables para parecer más humano",
            "✅ Limitar requests concurrentes (máximo 1-2 por dominio)",
            "✅ Implementar backoff exponencial en errores",
            "✅ Monitorear tiempo de respuesta del servidor",
            "✅ Pausar o reducir velocidad si el servidor está lento",
            "✅ Evitar hacer scraping en horas pico del sitio"
        ],
        
        "🆔 Identificación y Headers": [
            "✅ Usar User-Agent identificable con información de contacto",
            "✅ Incluir headers HTTP realistas",
            "✅ Rotar User-Agents si es necesario (pero mantener identificación)",
            "✅ Incluir Accept, Accept-Language y otros headers comunes",
            "✅ Evitar headers que revelen herramientas de scraping"
        ],
        
        "🔧 Aspectos Técnicos": [
            "✅ Manejar errores gracefully (404, 500, timeouts)",
            "✅ Implementar reintentos con backoff",
            "✅ Usar timeouts apropiados para requests",
            "✅ Manejar diferentes encodings de texto",
            "✅ Limpiar y validar datos extraídos",
            "✅ Implementar detección de cambios en estructura",
            "✅ Usar proxies rotativos si es necesario"
        ],
        
        "📊 Monitoreo y Logging": [
            "✅ Loggear todos los requests realizados",
            "✅ Monitorear tasas de error y bloqueos",
            "✅ Implementar alertas para problemas",
            "✅ Registrar métricas de rendimiento",
            "✅ Mantener audit trail de todas las actividades",
            "✅ Generar reportes de cumplimiento regulares"
        ],
        
        "🛡️ Seguridad y Privacidad": [
            "✅ No almacenar credenciales en código",
            "✅ Usar HTTPS cuando esté disponible",
            "✅ Anonimizar datos personales si se encuentran",
            "✅ Implementar cifrado para datos sensibles",
            "✅ Seguir principios de minimización de datos",
            "✅ Implementar políticas de retención de datos"
        ],
        
        "💾 Almacenamiento y Procesamiento": [
            "✅ Evitar duplicados de datos",
            "✅ Implementar versionado de datos",
            "✅ Documentar esquema de datos",
            "✅ Implementar backups regulares",
            "✅ Optimizar consultas de base de datos",
            "✅ Limpiar datos obsoletos regularmente"
        ],
        
        "🚀 Mantenimiento y Escalabilidad": [
            "✅ Monitorear cambios en sitios web target",
            "✅ Mantener código modular y configurable",
            "✅ Implementar tests automatizados",
            "✅ Documentar configuración y uso",
            "✅ Planificar escalabilidad horizontal",
            "✅ Implementar métricas de negocio relevantes"
        ]
    }
    
    return checklist

def print_checklist():
    """Imprimir checklist formateado"""
    checklist = generate_best_practices_checklist()
    
    print("📋 CHECKLIST COMPLETO DE MEJORES PRÁCTICAS")
    print("=" * 60)
    
    for category, items in checklist.items():
        print(f"\n{category}")
        print("-" * len(category))
        
        for item in items:
            print(f"  {item}")
    
    total_items = sum(len(items) for items in checklist.values())
    print(f"\n📊 Total de mejores prácticas: {total_items}")
    
    return checklist

# Generar y mostrar checklist
practices_checklist = print_checklist()

# Función para evaluar cumplimiento
def evaluate_project_compliance(project_checklist: Dict[str, bool]) -> Dict[str, any]:
    """Evaluar cumplimiento de un proyecto"""
    
    total_practices = sum(len(items) for items in practices_checklist.values())
    completed_practices = sum(1 for completed in project_checklist.values() if completed)
    
    compliance_percentage = (completed_practices / total_practices) * 100
    
    # Determinar nivel de cumplimiento
    if compliance_percentage >= 90:
        compliance_level = "Excelente 🏆"
    elif compliance_percentage >= 75:
        compliance_level = "Bueno 👍"
    elif compliance_percentage >= 60:
        compliance_level = "Aceptable ⚠️"
    else:
        compliance_level = "Necesita Mejoras ❌"
    
    return {
        'compliance_percentage': compliance_percentage,
        'completed_practices': completed_practices,
        'total_practices': total_practices,
        'compliance_level': compliance_level,
        'missing_practices': total_practices - completed_practices
    }

print(f"\n\n💡 CONSEJOS FINALES:")
print("=" * 25)
print("🎯 El scraping ético no es solo sobre cumplir reglas")
print("🤝 Es sobre ser un buen ciudadano digital")
print("⚖️ Siempre pregúntate: ¿Es esto lo correcto?")
print("📚 Mantente actualizado sobre cambios legales")
print("🔄 Revisa y actualiza prácticas regularmente")
print("🌍 Considera el impacto en la comunidad web")

## 🎓 Graduación: Eres Ahora un Web Scraper Ético

## 🏆 ¡Felicidades! Has Completado el Curso

### 🎯 Lo que Has Dominado

1. **Fundamentos Sólidos**:
   - HTML, CSS, y estructura web
   - HTTP, requests, y manejo de respuestas
   - Beautiful Soup y parsing de documentos

2. **Técnicas Avanzadas de Selección**:
   - XPath completo con predicados y funciones
   - CSS Selectors avanzados
   - Expresiones regulares para patrones complejos

3. **Scrapy Framework Master**:
   - Spiders básicos y avanzados
   - CrawlSpiders y seguimiento de enlaces
   - Pipelines y transformación de datos
   - Middlewares personalizados

4. **Procesamiento de Datos Industrial**:
   - Limpieza y validación automática
   - Bases de datos y almacenamiento
   - Análisis y visualización
   - Sistemas de monitoreo

5. **Ética y Responsabilidad**:
   - Aspectos legales del web scraping
   - Robots.txt y políticas de cortesía
   - Rate limiting inteligente
   - Frameworks de scraping ético

### 🛠️ Tu Arsenal de Herramientas

```python
# Toolkit Completo del Web Scraper Ético
requests                    # HTTP requests
beautifulsoup4             # HTML parsing
scrapy                     # Framework industrial
lxml                       # XPath avanzado
pandas                     # Procesamiento datos
sqlite3                    # Base de datos
matplotlib/seaborn         # Visualización
selenium                   # JavaScript handling
urllib.robotparser         # Robots.txt
```

### 📈 Niveles de Expertise Alcanzados

- **🟢 Principiante → Superado**: HTML básico, requests simples
- **🟡 Intermedio → Superado**: Beautiful Soup, CSS selectors, manejo de errores
- **🟠 Avanzado → Superado**: XPath, Scrapy, pipelines, bases de datos
- **🔴 Experto → ¡ALCANZADO!**: Frameworks éticos, sistemas industriales, monitoreo

### 🚀 Próximos Pasos en tu Carrera

1. **Proyectos Personales**:
   - Construir scrapers para tus intereses
   - Crear dashboards de datos
   - Automatizar tareas repetitivas

2. **Especialización Avanzada**:
   - Machine Learning para análisis de datos
   - APIs y microservicios
   - Cloud computing y escalabilidad
   - Real-time data processing

3. **Contribución a la Comunidad**:
   - Contribuir a proyectos open source
   - Compartir conocimiento en blogs/tutorials
   - Mentorizar otros developers

### 🌟 Principios para Recordar Siempre

1. **🛡️ Ética Primero**: Siempre considera el impacto de tu scraping
2. **🤖 Respeta Robots.txt**: Es la ley no escrita de la web
3. **⏱️ Sé Cortés**: Rate limiting salva servidores y tu reputación
4. **📊 Calidad sobre Cantidad**: Datos limpios valen más que datos abundantes
5. **🔄 Mantén y Actualiza**: El web cambia, tu código también debe hacerlo
6. **📚 Nunca Pares de Aprender**: La tecnología evoluciona constantemente

### 🎖️ Certificado de Completación

```
🏆 CERTIFICADO DE MAESTRÍA EN WEB SCRAPING ÉTICO 🏆

Por la presente se certifica que has completado exitosamente
el Curso Completo de Web Scraping con Python

Habilidades Certificadas:
✅ Web Scraping Ético y Responsable
✅ Frameworks Scrapy Avanzado
✅ Procesamiento Industrial de Datos
✅ Compliance Legal y Técnico
✅ Arquitectura de Sistemas Escalables

Fecha: {datetime.now().strftime('%B %d, %Y')}
Instructor: Claude (Anthropic)
Curso: Web Scraping con Python - Nivel Experto
```

### 💌 Mensaje Final

Has recorrido un camino increíble desde los conceptos básicos de HTML hasta la construcción de sistemas completos de scraping ético. No solo has aprendido a extraer datos de la web, sino que has desarrollado una mentalidad responsable y profesional hacia esta poderosa tecnología.

El web scraping es una herramienta que puede transformar industrias, acelerar investigación, y democratizar el acceso a información. Con los conocimientos que ahora posees, tienes la responsabilidad de usar esta herramienta de manera ética y constructiva.

### 🌐 La Web Te Espera

Ahora ve y construye cosas increíbles. El mundo de los datos está esperando a que lo explores de manera responsable y ética.

**¡Que tengas un excelente viaje scrapeando! 🕷️✨**

---

*"Con gran poder viene gran responsabilidad"* - Aplica esto siempre a tu web scraping. 🛡️